# 03 功能像预处理

这是功能像处理的第一部分, 从前面的预处理, 一直到功能像到结构像的配准

In [12]:
#导入预处理用的RZutilpy包
%cd /home/yufengxia/Code
from RZutilpy.system import Path, unix_wrapper, gettimestr, makedirs
from RZutilpy.rzio import matchfiles
from RZutilpy.figure import plot
from RZutilpy.mri import findminoutlier
import os
import numpy as np

/home/yufengxia/Code


In [6]:
subj = 'M009'

# a list of functional NIFTI files
files = matchfiles(f'/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/*mbep2d*.nii')

files

['/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_2_cmrr_mbep2d_REST-3mmISO_PA_300.nii',
 '/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_3_cmrr_mbep2d_REST-3mmISO_PA_300.nii',
 '/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_8_cmrr_mbep2d_TASK1-1-3mmISO_PA.nii',
 '/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_9_cmrr_mbep2d_TASK1-1-3mmISO_PA.nii',
 '/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_10_cmrr_mbep2d_TASK1-2-3mmISO_PA.nii',
 '/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_11_cmrr_mbep2d_TASK1-2-3mmISO_PA.nii',
 '/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_34_cmrr_mbep2d_3mmISO.nii',
 '/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_35_c

In [15]:
# subject name
subj = 'M009'

# a list of functional NIFTI files
files = matchfiles(f'/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/*mbep2d*.nii')
for i in range(len(files)-2,-1,-2):
    del files[i]
    
# t1 file
t1 = Path(f'/home/software/freesurfer/7.3.2/subjects/{subj}/SUMA/{subj}_SurfVol.nii')
t1ss = Path(f'/home/software/freesurfer/7.3.2/subjects/{subj}/mni/anatSS.{subj}.nii')

# output directory, if exists, we exit
output_dir = Path(f'/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/funcpp/')

# number of tr to discard
tr_discard = 0  # number of tr to discard

# motion censor limit
motion_censor = 0.3  # threshold for motion censoring, default:(0.3)

# extra option for align_epi_anat.py
align_opt = ['-giant_move']

fwdfiles = ['01', '02', '03'] # 这里需要手动run的顺序，用来做配对distortion correction
revfiles = ['04', '04', '04']
# some calculation
nRuns = len(files)
runstr = [f'{i+1:02d}' for i in range(nRuns)]
motion_censor = 0.3 if motion_censor is None else motion_censor

In [16]:
files

['/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_3_cmrr_mbep2d_REST-3mmISO_PA_300.nii',
 '/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_9_cmrr_mbep2d_TASK1-1-3mmISO_PA.nii',
 '/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_11_cmrr_mbep2d_TASK1-2-3mmISO_PA.nii',
 '/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_35_cmrr_mbep2d_3mmISO.nii']

-----------------华丽的分界线-----------------

在这一部分，我们主要做到以下几件事情
1. 把所有的功能像数据先copy到一个<output_dir>文件夹下, 避免修改原始的功能像数据, 然后去掉每个run前面的几个TR <tr_discard>
2. 矫正epi的朝向和正中点
    其中如果epi和T1是在不同的session采集的，那么可能会有比较大的偏差。需要手动的移动epi的图像到和用来做FreeSurfer的t1像一致
3. 到<output_dir> 文件夹下面，找到头动最小的volume，然后把这个作为头动矫正的基准。这一步会检查头动，并且记录一些特别大头动的volume，以头动超过一定的threshold为准

## Step 1. 先需要做一些文件检查

In [34]:
# ============== deal with some input parameters =======================
# Show some diagnostic information
# check file exists
%cd /home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/
for i in files:
    assert Path(i).exists(), f'{i} does not exist!'

t1 = Path(t1)
assert t1.exists(), 'T1 file does not exist!'

# generate slice timing file
if not Path('SliceTiming.txt').exists():
    import json 
    with open(f'{Path(files[0]).pstem}.json') as f:
        jsoninfo = json.load(f)
    np.savetxt('SliceTiming.txt', jsoninfo['SliceTiming'])
    print('generate SliceTiming.txt!')
    del jsoninfo

# 整理forward和reverse的文件，如果两者数量不相等，我们自动补齐缺数量
if len(fwdfiles)!=len(fwdfiles):
    if len(fwdfiles)>len(revfiles):
        nPair = len(fwdfiles)
        revfiles = revfiles + [revfiles[end]]*(nPair-len(revfiles))
    else:
        nPair = len(revfiles)
        fwdfiles = fwdfiles + [fwdfiles[end]]*(nPair-len(fwdfiles))

# print out some diagnoistic
print(f'\nt1 file is: \n{t1}')
print(f'functional files are: \n')
[print(f'{i}') for i in files]
print(f'\nforward files are: {fwdfiles} \n')
print(f'reverse files are: {revfiles} \n')
print(f'\nslice timing file is: \nSliceTiming.txt')

/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009

t1 file is: 
/home/software/freesurfer/7.3.2/subjects/M009/SUMA/M009_SurfVol.nii
functional files are: 

/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_2_cmrr_mbep2d_REST-3mmISO_PA_300.nii
/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_3_cmrr_mbep2d_REST-3mmISO_PA_300.nii
/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_8_cmrr_mbep2d_TASK1-1-3mmISO_PA.nii
/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_9_cmrr_mbep2d_TASK1-1-3mmISO_PA.nii
/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_10_cmrr_mbep2d_TASK1-2-3mmISO_PA.nii
/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/20230312112243_11_cmrr_mbep2d_TASK1-2-3mmISO_PA.nii

forward files are: ['01', '03', '05'] 

reverse files are:

## Step 2. 进一步的设置

In [35]:
start_time = gettimestr("full")
print(f'\n=============== Preprocessing started: {start_time} ================\n')
# change shell to tcsh, afni default shell is tcsh
orig_shell = os.environ['SHELL']
os.environ['SHELL']='/usr/bin/tcsh'
cwd = Path.cwd() # record current directory, we will go back

# verify that the results directory does not yet exist
output_dir = Path(output_dir)
assert not output_dir.exists(), f'output dir {output_dir} already exists'
makedirs(output_dir)
#makedirs((output_dir / 'stimuli'))
# enter the results directory (can begin processing data)
os.chdir(output_dir)
# copy anatomy to results dir
! 3dcopy {t1} {t1.pstem}
t1 = Path(f'{t1.pstem}+orig')  # switch t1 to the new location

# copy nonlinear-warping MNI files to this folder
FREESURFER_HOME = os.getenv('FREESURFER_HOME')
AFNI_HOME = os.getenv('AFNIDIR')
! cp {FREESURFER_HOME}/subjects/{subj}/mni/anatQQ.{subj}_WARP.nii ./
! cp {FREESURFER_HOME}/subjects/{subj}/mni/anatQQ.{subj}.aff12.1D ./
! cp {AFNI_HOME}/MNI152_2009_template_SSW.nii.gz ./

# 利用fslreorient2std把朝向划到和MNI一样
for file in files:
    ! fslreorient2std {file}


=============== Preprocessing started: 2023-05-09, 19:19:34 ================

++ 3dcopy: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1


## Step 3. 复制文件

复制文件到<output_dir>, 同时去掉最开始的几个TR

In [36]:
# ============================ auto block: tcat ============================
# apply 3dTcat to copy input dsets to results dir,
# while might want removing the first TRs
for file, run in zip(files, runstr):
    cmd=f'3dTcat -prefix pb00.{subj}.r{run}.tcat {file}[{tr_discard}..$]'
    unix_wrapper(cmd)

++ 3dTcat: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
++ elapsed time = 0.0 s
++ 3dTcat: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
++ elapsed time = 0.6 s
++ 3dTcat: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
++ elapsed time = 0.0 s
++ 3dTcat: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
++ elapsed time = 0.4 s
++ 3dTcat: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
++ elapsed time = 0.0 s
++ 3dTcat: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
++ elapsed time = 0.4 s


## Step 4. 移动EPI文件使其和T1中心重合

In [37]:
# ============================auto block: giant_move, added by RZ ===========================
for run in runstr:
    # 把图像的中点划到统一显示空间的中点
    ! 3drefit -deoblique -xorigin cen -yorigin cen -zorigin cen pb00.{subj}.r{run}.tcat+orig
    # 然后把function数据和 t1ss文件的重心移动到一致，这样有利于进行配准
    ! @Align_Centers -cm -no_cp -base {t1ss} -dset pb00.{subj}.r{run}.tcat+orig

! rm *_shft.1D

++ 3drefit: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset pb00.M009.r01.tcat+orig
 + deoblique
++ 3drefit processed 1 datasets
++ Rewriting header ./pb00.M009.r01.tcat+orig.HEAD
*+ WARNING: Over-writing dataset ./pb00.M009.r01.tcat+orig.HEAD
base: 1.509 21.3561 9.12111
dset: 0.0334702 1.18719 11.5481
delta: 1.475530 20.168910 -2.426990
++ 3drefit: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset pb00.M009.r02.tcat+orig
 + deoblique
++ 3drefit processed 1 datasets
++ Rewriting header ./pb00.M009.r02.tcat+orig.HEAD
*+ WARNING: Over-writing dataset ./pb00.M009.r02.tcat+orig.HEAD
base: 1.509 21.3561 9.12111
dset: -0.0851974 1.27687 11.8872
delta: 1.594197 20.079230 -2.766090
++ 3drefit: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset pb00.M009.r03.tcat+orig
 + deoblique
++ 3drefit processed 1 datasets
++ Rewriting header ./pb00.M009.r03.t

 ## Step 5. 同时找到outlier最小的一个volume作为后面motion correction的base

In [8]:
%cd '/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/funcpp/'
findminoutlier(f'pb00.{subj}.r*.tcat+orig.HEAD', output_dir)
# this part will generate several files
#   out.pre_ss_warn.txt: warning for pre-steady state in the first TRs, consider change tr_discard
#   outcont.r**.1D: fraction of outlier in each volume
#   outcount_rall.1D concatenate fraction of outlier
#   out.min_outlier.txt  tells you which run, which TR is min_outlier

/home/yufengxia/Code/yfXRepo/fMRICourse/Example_data/20230312_BPhappy_M009/funcpp
++ 3dToutcount: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
++ 45637 voxels passed mask/clip
++ 3dToutcount: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
++ 45945 voxels passed mask/clip
++ 3dToutcount: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
++ 45845 voxels passed mask/clip
++ 3dToutcount: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
++ 48344 voxels passed mask/clip
++ 3dbucket: AFNI version=AFNI_22.2.10 (Sep  2 2022) [64-bit]
min outlier: run 1, TR 37


In [4]:
Path.cwd()

Path('/home/yufengxia/Code')